In [140]:
import pandas as pd
from kafka import KafkaProducer
from time import sleep
from json import dumps
import dask.dataframe as dd

In [141]:
producer = KafkaProducer(
    bootstrap_servers=['51.20.250.74:9092'], 
    value_serializer=lambda x: dumps(x).encode('utf-8')
)

In [142]:
ddf = dd.read_csv("indexProcessed.csv")

In [143]:
# Define Map function
def map_function(batch_df):
    # Extract relevant information from the batch
    date = batch_df['Date']
    close_price = batch_df['Close']
    volume = batch_df['Volume']

    # Emit intermediate key-value pairs
    return (date.compute().iloc[0], {
        'mean_close': close_price.mean().compute(),
        'min_close': close_price.min().compute(),
        'max_close': close_price.max().compute(),
        'std_close': close_price.std().compute(),
        'mean_volume': volume.mean().compute()
    })

In [144]:
# Process data in batches
batch_size = 1000
start_index = 0

In [145]:
while start_index < len(df):
    end_index = min(start_index + batch_size, len(df))
    # Use loc for row selection
    batch_df = ddf.loc[start_index:end_index]

    # Map Phase: Apply map function to the batch
    intermediate_data = map_function(batch_df)

    # Shuffling and Sorting (not explicitly done since data is not distributed)

    # Reduce Phase: Aggregate the data for each date
    final_result = {intermediate_data[0]: intermediate_data[1]}

    # Send the final result to Kafka
    producer.send('demo_test', value=final_result)

    start_index += batch_size
    sleep(1)

KeyboardInterrupt: 

In [127]:
# Flush the producer (optional, but good practice)
producer.flush()

In [31]:
df = pd.read_csv("indexProcessed.csv")


In [32]:
df.head()

,Index,Date,Open,High,Low,Close,Adj Close,Volume,CloseUSD
0,HSI,1986-12-31,2568.300049,2568.300049,2568.300049,2568.300049,2568.300049,0.0,333.879006
1,HSI,1987-01-02,2540.100098,2540.100098,2540.100098,2540.100098,2540.100098,0.0,330.213013
2,HSI,1987-01-05,2552.399902,2552.399902,2552.399902,2552.399902,2552.399902,0.0,331.811987
3,HSI,1987-01-06,2583.899902,2583.899902,2583.899902,2583.899902,2583.899902,0.0,335.906987
4,HSI,1987-01-07,2607.100098,2607.100098,2607.100098,2607.100098,2607.100098,0.0,338.923013


In [33]:
df.sample(1)

,Index,Date,Open,High,Low,Close,Adj Close,Volume,CloseUSD
21467,NYA,2017-07-20,11956.26953,11963.54981,11928.66992,11944.49023,11944.49023,3.182780e+09,11944.49023


In [37]:
while True:
    dict_stock = df.sample(1).to_dict(orient="records")[0]
    producer.send('demo_test', value=dict_stock)
    sleep(2)

KeyboardInterrupt: 

In [54]:
producer.flush() #clear data from kafka server